In [ ]:
import numpy as np
import sep
#plt.savefig("MyFigure.png", format='png')pdf after each image
#replicate the code with the new "image"
#line 15 "print length of obejcts" 
#histogram flux - histogram 
#matplotlib.pyplot.hist()
#numpy function for all of them

#the biggest diffrence bewteen the flux point and the mean value
#new list thediffrence = flux list - mean
#numpy.argmax
#plot it by - line 14 len(object) put in only one [i] for the outlier 

#???
#tuesday class point 8 

In [ ]:
import astropy
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

In [ ]:
# read image into standard 2-d numpy array
data = astropy.io.fits.get_data("image.fits")

In [ ]:
# show the image
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.savefig("MyFigure.png", format='png')

**Background subtraction**

In [ ]:
# measure a spatially varying background on the image
bkg = sep.Background(data)

In [ ]:
bkg = sep.Background(data, mask=mask, bw=64, bh=64, fw=3, fh=3)

In [ ]:
# get a "global" mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
# evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()
# bkg_image = np.array(bkg) # equivalent to above

In [ ]:
# show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig("MyFigure.png", format='png')

In [ ]:
# evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()

In [ ]:
# show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig("MyFigure.png", format='png')

In [ ]:
# subtract the background
data_sub = data - bkg

**Object detection**

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

In [ ]:
# how many objects were detected
len(objects)

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
plt.savefig("MyFigure.png", format='png')

In [ ]:
# available fields
objects.dtype.names

**Aperture photometry**

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))